In [67]:
import pyupbit
import time
from datetime import datetime, timedelta
from pytz import timezone, utc
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import trange, notebook
pd.options.display.max_rows = 1441

# KST = timezone('Asia/Seoul')
# date = KST.localize(datetime.now())
# yesterday = date - timedelta(1)
# yesterday = yesterday.strftime("%Y-%m-%d")
# today = (KST.localize(datetime.now())).strftime("%Y-%m-%d")

def get_rate(p1, p2):
    return round(((p1/p2 - 1) * 100), 2)

def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

def generate_pick_coins(date):
    coins = pyupbit.get_tickers(fiat="KRW")
    coins_rate = []
    for coin in coins:
        coin_price = pyupbit.get_ohlcv(ticker=coin, to=date, count=1)
        time.sleep(0.1)
        open_price = coin_price.open.values[0]
        close_price = coin_price.close.values[0]
        coin_rate = get_rate(close_price, open_price)
        if coin_rate > 5:
            yield [date, coin, coin_rate]

def get_pick_coins(date):
#     for i in generate_pick_coins(date):
#         print(i)
    picked_coins = sorted(generate_pick_coins(date), key = lambda x : -x[2])[0:5]
    return picked_coins

def get_missing_value_frame(df, date):
    tomorrow = get_tomorrow(date)
    df["DATE"] = pd.to_datetime(df.index)
    df = df[f'{date} 09:00:00':f'{tomorrow} 09:00:00']
    df = df.resample(rule='min').last()
    df = df.fillna(method='ffill')
    return df

def get_coin_data(coins_name):
    for date, coin in coins_name:
        coin_dict = {}
        coin_dict["name"] = coin
        coin_dict["data"] = get_missing_value_frame(pyupbit.get_ohlcv(coin, count=1441, 
                                            to=datetime.strptime(f"{date} 9:01:00", "%Y-%m-%d %H:%M:%S")
                                            + timedelta(1) ,interval="minute1"), date) 
        
        coin_dict['open'] = coin_dict["data"].iloc[0]['open']
        coin_dict['close'] = coin_dict["data"].iloc[-1]['close']
        yield coin_dict
        
def get_tomorrow(date:str) -> str:
    tomorrow = datetime.strptime(date, '%Y-%m-%d') + timedelta(1)
    return tomorrow.strftime('%Y-%m-%d')

def get_avg(result):
    mount = 0
    for rate in result:
        a = list(rate.values())
        mount += a[0][0]
    avg = mount / len(result)

    return avg

def backtest(date):
    pick_coins = get_pick_coins(date)
    coins_name = [(coin[0], coin[1]) for coin in pick_coins]
    return_rate = []
    for price in get_coin_data(coins_name):
        coin_dict = {}
        coin_dict[price["name"]] = (0, 0)
        check = None

        for idx, p in price["data"].iterrows():
            
            if get_rate(p[3], price['open']) <= -3:
                coin_dict[price["name"]] = (-3, 1) # 손절
                return_rate.append(coin_dict)
                print(coin_dict)
                break

            if get_rate(p[3], price['open']) > 10:
                check = "half" 

            if check == "half" and get_rate(p[3], price['open']) <= 0:
                coin_dict[price["name"]] = (5, 2)  # 경우 2
                return_rate.append(coin_dict)
                print(coin_dict)
                break

            if check == "half" and get_rate(p[3], price['open']) > 50:
                coin_dict[price["name"]] = (37, 3) # 경우 1
                return_rate.append(coin_dict)
                print(coin_dict)
                break
                

            if idx == f"{get_tomorrow(date)} 09:00:00" and check == "half": # 경우 3
                coin_dict[price["name"]] = (5 + 0.55 * get_rate(price['close'], price['open']), 4)
                return_rate.append(coin_dict)
                print(coin_dict)
                break

            if idx == pd.to_datetime(f"{get_tomorrow(date)} 09:00:00"): # 경우 4
                coin_dict[price["name"]] = (get_rate(price['close'], price['open']), 5)
                return_rate.append(coin_dict)
                print(coin_dict)
                break
                
    rr = 0
    if len(return_rate):
        rr = get_avg(return_rate) * (len(return_rate) / 5)

    return rr

In [76]:
result = []
for date in date_range("2021-08-27", "2021-09-14"):
    result.append([date, backtest(date)])
    
df = pd.DataFrame(result, columns=['Date', 'return_rate'])
df["dd"] = ((100 + df['return_rate'])/100)
df["crr"] = df["dd"].cumprod()

{'KRW-HIVE': (-3, 1)}
{'KRW-WAXP': (-3, 1)}
{'KRW-BCHA': (-3, 1)}
{'KRW-XTZ': (-3, 1)}
{'KRW-SRM': (10.26, 5)}
{'KRW-ADA': (-3, 1)}
{'KRW-ATOM': (-3, 1)}
{'KRW-BCHA': (0.98, 5)}
{'KRW-BSV': (7.27, 5)}
{'KRW-BTG': (0.45, 5)}
{'KRW-SRM': (-3, 1)}
{'KRW-XTZ': (-3, 1)}
{'KRW-SAND': (5, 2)}
{'KRW-FLOW': (-3, 1)}
{'KRW-STMX': (-3, 1)}
{'KRW-MANA': (-3, 1)}
{'KRW-BSV': (-3, 1)}
{'KRW-PLA': (-3, 1)}
{'KRW-FCT2': (-3, 1)}
{'KRW-ORBS': (-0.74, 5)}
{'KRW-WAVES': (-3, 1)}
{'KRW-BCHA': (5, 2)}
{'KRW-MTL': (-3, 1)}
{'KRW-DOT': (-3, 1)}
{'KRW-ELF': (-3, 1)}
{'KRW-GRS': (5, 2)}
{'KRW-XRP': (2.89, 5)}
{'KRW-AHT': (-3, 1)}
{'KRW-ELF': (-3, 1)}
{'KRW-VET': (4.4, 5)}
{'KRW-ETH': (-0.47, 5)}
{'KRW-LINK': (1.25, 5)}
{'KRW-BCHA': (37, 3)}
{'KRW-BSV': (5, 2)}
{'KRW-GLM': (13.95, 5)}
{'KRW-BTT': (2.19, 5)}
{'KRW-JST': (3.01, 5)}
{'KRW-POLY': (37, 3)}
{'KRW-BCHA': (-3, 1)}
{'KRW-ELF': (5.1, 5)}
{'KRW-HIVE': (37, 3)}
{'KRW-IOST': (13.76, 5)}
{'KRW-CVC': (-3, 1)}
{'KRW-STMX': (-3, 1)}
{'KRW-ARDR': (37, 3)}
{'KRW-

,Date,return_rate,dd,crr
0,2021-08-27,-1.200,0.98800,0.988000
1,2021-08-28,-0.348,0.99652,0.984562
2,2021-08-29,0.540,1.00540,0.989878
3,2021-08-30,-1.400,0.98600,0.976020
4,2021-08-31,-0.948,0.99052,0.966767
5,2021-09-01,-0.222,0.99778,0.964621
6,2021-09-02,-0.164,0.99836,0.963039
7,2021-09-03,12.230,1.12230,1.080819
8,2021-09-04,17.972,1.17972,1.275064
9,2021-09-05,6.600,1.06600,1.359218


In [ ]:
# 매도 알고리즘
##### 폭락장에서는 사지 않는다. 

# -3퍼 아래 손절
# 저가 low가 -3퍼 아래 손절
# 저가가 고가보다 빠른 시간에 형성될 경우 -3퍼로 계산
# 경우1: -3퍼가 형성되기 이전에 10% 일 경우 절반 5퍼 + 50% 넘어갈 경우 전량매도 -> 37퍼
# 경우2: -3퍼가 형성되기 이전에 10% 절반 익절 후 0퍼 수익률로 갔을 경우 -> 5퍼
# 경우3: -3퍼까지 가지 않았으며 10% 아래 + 수익률일 경우 -> 그날의 종가 수익률 -> 5퍼 + 5.5 * 종가 수익률 
# 경우4: -3퍼가 형성되기 이전에 10% 절반 익절 후 0퍼 수익률로 가지 않을 경우 -> 그날의 종가 수익률
# 경우5(고려 사항): 특정 시간 이후로 -3%까지는 아니지만 - 수익률일 경우 절반 손절


In [ ]:
def mean(nums):
    _sum = 0
    for i in nums:
        _sum += i
        
    return _sum / len(nums)

print(mean([4,5,6])) #5
print(mean([1,2,3,4,5])) #3

means2 = lambda *args: sum(args)/len(args) [4,5,6]



print(means2(4,5,6)) #5